# Simple networks

### Useful Links:

* pytorch official documentation
http://pytorch.org/docs/master/index.html

* pytorch discussion
https://discuss.pytorch.org/

* pytorch official tutorials
https://pytorch.org/tutorials/

## Preliminaries

In [ ]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# to make interactive plotting possible
%matplotlib inline
# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

In [ ]:
# make plots a bit nicer
plt.matplotlib.rcParams.update({'font.size': 18, 'font.family': 'serif'})

In [ ]:
# random seed settings
torch.manual_seed(42)
np.random.seed(42)

# data type (useful to have in pytorch)
dtype_np = np.float64
dtype_torch = torch.FloatTensor

In [ ]:
# print versions
from platform import python_version
print(f"python version: {python_version()}")
print(f"torch version: {torch.__version__}")

### Pathes

In [ ]:
# transfer data to colab
!wget https://raw.githubusercontent.com/lyubonko/ucu2021cv/master/assignments/data/toy_data/data_class_train.txt
!wget https://raw.githubusercontent.com/lyubonko/ucu2021cv/master/assignments/data/toy_data/data_class_test.txt

# create local folder with data
path_folder = "data"
if not os.path.exists(path_folder):
  os.mkdir(path_folder)

# data pathes
path_data_train = os.path.join(path_folder, "data_class_train.txt")
path_data_test = os.path.join(path_folder, "data_class_test.txt")

# move files 
os.replace("data_class_test.txt", path_data_test)
os.replace("data_class_train.txt", path_data_train)

### Data (playground)

In [ ]:
# we will work with one file (but we can do it with two files)
path_data = path_data_train

In [ ]:
# load data
data = np.loadtxt(path_data)
dataX = data[:,0:2]
dataY = data[:,2]

In [ ]:
n_samples = data.shape[0]
dim_in = 2 # two features
dim_out = 3 # three classes

n_train = int(n_samples * 0.7)
n_test = n_samples - n_train

In [ ]:
# train-test partition
perm = np.random.permutation(n_samples)
train_indx = perm[:n_train]
test_indx = perm[n_train:]

dataX_train, dataY_train = dataX[train_indx,:], dataY[train_indx]
dataX_test, dataY_test = dataX[test_indx,:], dataY[test_indx]

In [ ]:
# visualize data
plt.figure(figsize=(10,5))
plt.plot(dataX_train[dataY_train==0,0], dataX_train[dataY_train==0,1],'ob', label="class1")
plt.plot(dataX_train[dataY_train==1,0], dataX_train[dataY_train==1,1],'og', label="class2")
plt.plot(dataX_train[dataY_train==2,0], dataX_train[dataY_train==2,1],'or', label="class3")

plt.plot(dataX_test[:,0], dataX_test[:,1],'xk', label="test")

plt.xlabel('feature #1')
plt.ylabel('feature #2')
plt.legend()
plt.xlim(-9, 9)
plt.ylim(-5, 12);

## Two-layer Network ('by hand')


$$x_{hidden} = RELU(x  \cdot W_1 + b_1)$$
$$y_{pred} = x_{hidden} \cdot W_2 + b_2$$

In [ ]:
dim_hidden = 100 # hidden dimension

In [ ]:
# input 
x = torch.from_numpy(dataX_train).type(dtype_torch)
y = torch.from_numpy(dataY_train).type(torch.LongTensor)

* Weights and biases

### <font color="red"> **[PROBLEM I]**: </font>   
 <font color="red"> Fill the missing part (weights and biases for the output layer) </font>  

In [ ]:
# Randomly initialize weights
w1_value = np.random.randn(dim_in, dim_hidden)
w2_value = None # YOUR CODE HERE

# Randomly initialize biases
b1_value = np.random.randn(dim_hidden)
b2_value = None # YOUR CODE HERE


* tell pytorch that these variables will participate in backprop. requires_grad=False by default, more details: https://pytorch.org/docs/stable/notes/autograd.html

In [ ]:
w1 = torch.from_numpy(w1_value).type(dtype_torch).requires_grad_(True)
w2 = torch.from_numpy(w2_value).type(dtype_torch).requires_grad_(True)
b1 = torch.from_numpy(b1_value).type(dtype_torch).requires_grad_(True)
b2 = torch.from_numpy(b2_value).type(dtype_torch).requires_grad_(True)

* Loss (we will use cross-entropy loss), see documentation for details http://pytorch.org/docs/master/nn.html#torch.nn.CrossEntropyLoss

In [ ]:
criterion = nn.CrossEntropyLoss()

* learning parameters

In [ ]:
learning_rate = 1e-3
n_iteration = 1000

### <font color="red"> **[PROBLEM II]**: </font>   
 <font color="red"> Fill the missing part (last operation in forward pass to calculate *y_pred*) </font>  

In [ ]:
logger = {}
logger['iteration'] = []
logger['loss_iteration'] = []

for t in range(n_iteration):  
 
    # forward pass
    x_hidden = x.mm(w1) + b1
    x_hidden_act = x_hidden.clamp(min=0) # apply RELU
    y_pred = None # YOUR CODE HERE
    
    # compute loss
    loss = criterion(y_pred, y)

    # backprop
    loss.backward()

    # update weights using gradient descent  
    w1.data -= learning_rate * w1.grad
    w2.data -= learning_rate * w2.grad
    b1.data -= learning_rate * b1.grad
    b2.data -= learning_rate * b2.grad

    # manually zero the gradients
    w1.grad.zero_()
    w2.grad.zero_()
    b1.grad.zero_()
    b2.grad.zero_()      
    
    # reporting & logging       
    if t % 100 == 0:
        print(t, loss.item())
        
    logger['iteration'] += [t]
    logger['loss_iteration'] += [loss.item()]

 <font color="green"> After visualizing the loss (cell below) you should see something like this </font>

![loss_toy.png](https://raw.githubusercontent.com/lyubonko/ucu2021cv/master/assignments/fig/loss_toy.png)

In [ ]:
# visualize loss
plt.figure(figsize=(10,5))
plt.plot(logger['iteration'], logger['loss_iteration'],'ob', label="loss")

plt.xlabel('iteration')
plt.ylabel('loss');

### <font color="red"> **[PROBLEM III]**: </font>   
<font color="red"> Implement the fuction which takes x and predicts its class </font>  

In [ ]:
def predict(x, w1, b1, w2, b2, dtype_torch=torch.FloatTensor):
    """
    Prediction based on two-layer model (by hand)
    
    Args:
        x (numpy.array): sample
        w1, b1, w2, b2 (torch.Tensor) : weights and biases 
    Returns:
        scalar: predicted class
    """
    t = torch.from_numpy(x).type(dtype_torch)
    forward_pass = None # YOUR CODE HERE

    return np.argmax(forward_pass)

<span style="color:green"> by running the following command you should get number bigger than 8 </span>

In [ ]:
np.sum(np.equal([predict(x, w1, b1, w2, b2) for x in dataX_train[:10]], [0, 2, 0, 1, 0, 2, 0, 0, 1, 0]))

* Calculate accuracy

In [ ]:
def get_accuracy(y, y_pred):
    """
    Calculate accuracy given y and y_predicted
    
    Args:
        y (numpy.array): ground truth
        y_pred (numpy.array): predictated values
         
    Returns:
        scalar: accuracy
    """
    n_samples = y.shape[0]
    return np.sum(y == y_pred)/n_samples * 100

In [ ]:
y_train_predict = np.zeros(n_train)
for i in range(n_train):
    y_train_predict[i] = predict(dataX_train[i], w1, b1, w2, b2)

print("Train accuracy: %f" % get_accuracy(y_train_predict, dataY_train))

### <font color="red"> **[PROBLEM IV]**: </font>   
<font color="red"> Calculate accuracy on the test set </font>

In [ ]:
#YOUR CODE HERE
y_test_predict = None

* Visualize decision boundary

In [ ]:
Nspace = 100
x1space = np.linspace(-9, 9, Nspace)
x2space = np.linspace(-5, 12, Nspace)
X,Y = np.meshgrid(x1space, x2space)

Z = np.zeros((Nspace,Nspace))
for i in range(Nspace):
    for j in range(Nspace):
        x1 = x1space[j]
        y1 = x2space[i]
        x = np.array([x1,y1])
        Z[i,j] = predict(x, w1, b1, w2, b2)

plt.figure(figsize=(10,5))
plt.pcolor(X, Y, Z, vmin=abs(Z).min(), vmax=abs(Z).max())

plt.xlabel('feature #1')
plt.ylabel('feature #2')
plt.xlim(-9, 9)
plt.ylim(-5, 12);

## Two-layer Network, again ...  (with nn package) 

$$x_{hidden} = RELU(x  \cdot W_1 + b_1)$$
$$y_{pred} = x_{hidden} \cdot W_2 + b_2$$

After some hard work we will use torch with all its power and elegance.  

* Model

In [ ]:
class Net(nn.Module):
  
  def __init__(self, dim_in, dim_hidden, dim_out):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(dim_in, dim_hidden)
    self.fc2 = nn.Linear(dim_hidden, dim_out)
    
  def forward(self, x):
    x = self.fc2(F.relu(self.fc1(x)))
    return x

In [ ]:
model = Net(dim_in, dim_hidden, dim_out)

* Loss (we will use cross-entropy loss)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
# input 
x = torch.from_numpy(dataX_train).type(dtype_torch)
y = torch.from_numpy(dataY_train).type(torch.LongTensor)

In [ ]:
learning_rate = 1e-2
n_iteration = 1000

In [ ]:
logger = {}
logger['iteration'] = []
logger['loss_iteration'] = []

for t in range(n_iteration):  
    
    # forward pass
    y_pred = model(x)
    
    # compute loss
    loss = criterion(y_pred, y)

    # backprop
    loss.backward()

    # update weights using gradient descent  
    for param in model.parameters():
        param.data -= learning_rate * param.grad.data 
    
    # manually zero the gradients
    model.zero_grad()  
    
    # reporting & logging       
    if t % 100 == 0:
        print(t, loss.item())
        
    logger['iteration'] += [t]
    logger['loss_iteration'] += [loss.item()]

In [ ]:
def predict(x, model):
    """
    Prediction based on two-layer model
    
    Args:
        x (numpy.array): feature vector of a sample
         
    Returns:
        scalar: predicted class for this sample
    """    
    t = torch.from_numpy(x).type(dtype_torch)
    forward_pass = model(t)
    return np.argmax(forward_pass.data.numpy(), axis=1)

In [ ]:
y_train_predict = predict(dataX_train,model)
print("Train accuracy: %f" % get_accuracy(y_train_predict, dataY_train))

y_test_predict = predict(dataX_test, model)
print("Test accuracy: %f" % get_accuracy(y_test_predict, dataY_test))

### <font color="red"> **[PROBLEM V]**: </font>   
<font color="red"> Visualize loss </font>

In [ ]:
# YOUR CODE HERE